In [1]:
import os
from copy import deepcopy

import openmm as mm
from openmm import unit
from openmm import app
from pdbfixer import PDBFixer
import mdtraj as md
import nglview as nv
import requests
import numpy as np
from math import sqrt
from tqdm import tqdm
import matplotlib.pyplot as plt

# Playing with the potential energy

## Building the Barnase-Barstar complex

In [2]:
# Function to download PDB files

def fetch_pdb(pdb_id, download_path="./"):

        url = 'http://files.rcsb.org/download/{}.pdb'.format(pdb_id)
        try:
            res = requests.get(url, allow_redirects=True)
        except:
            print("Could not fetch pdb from {}".format(url))
            return 
        
        file_path = os.path.join(download_path, pdb_id + ".pdb")
        with open(file_path, "wb") as f:
            f.write(res.content)

In [3]:
# Just write the PDB id in order to download the pdb file

fetch_pdb("1brs")

In [4]:
# Load pdb with MDTraj

brs = md.load('1brs.pdb')

In [5]:
# Remove water molecules

brs = brs.remove_solvent()

In [6]:
# Barnase's chains

atoms_in_chain_A = brs.topology.select("chainid == 0")
atoms_in_chain_B = brs.topology.select("chainid == 1")
atoms_in_chain_C = brs.topology.select("chainid == 2")

In [7]:
# Barstar's chains

atoms_in_chain_D = brs.topology.select("chainid == 3")
atoms_in_chain_E = brs.topology.select("chainid == 4")
atoms_in_chain_F = brs.topology.select("chainid == 5")

In [8]:
# Barnase's chain's atoms

barnase_A = brs.atom_slice(atoms_in_chain_A)
barnase_B = brs.atom_slice(atoms_in_chain_B)
barnase_C = brs.atom_slice(atoms_in_chain_C)

In [9]:
# Barstar's chain's atoms

barstar_D = brs.atom_slice(atoms_in_chain_D)
barstar_E = brs.atom_slice(atoms_in_chain_E)
barstar_F = brs.atom_slice(atoms_in_chain_F)

Let's work with barnase B and barstar F since they are the most complete monomers. Given that barstar E is the bound to barnase B, barstar F needs to be fitted over E to have the complex B-F:

In [10]:
atoms_to_fit_E = []
atoms_to_fit_F = []
for atom_E in barstar_E.topology.atoms_by_name('CA'):
    for atom_F in barstar_F.topology.atoms_by_name('CA'):
        if str(atom_E)==str(atom_F):
            #print(f'{str(atom_E)} is {atom_E.index} in E and {atom_F.index} in F')
            atoms_to_fit_E.append(atom_E.index)
            atoms_to_fit_F.append(atom_F.index)

In [11]:
barstar_F_over_E = md.Trajectory.superpose(barstar_F, barstar_E, frame=0, atom_indices=atoms_to_fit_F, ref_atom_indices=atoms_to_fit_E)

In [12]:
new_barnase_barstar = barnase_B.stack(barstar_F_over_E)

In [13]:
view = nv.show_mdtraj(new_barnase_barstar)
view

NGLWidget()

In [14]:
new_barnase_barstar.save_pdb('new_barnase_barstar.pdb')

In [15]:
fixer = PDBFixer(filename='new_barnase_barstar.pdb')

In [16]:
fixer.findMissingResidues()
missing_residues = fixer.missingResidues
print(f"{len(missing_residues)} missing residues")

fixer.findNonstandardResidues()
nonstandard_residues = fixer.nonstandardResidues
print(f"{len(nonstandard_residues)} non standard residues")

fixer.findMissingAtoms()
missing_atoms = fixer.missingAtoms
missing_terminals = fixer.missingTerminals
print(f"{len(missing_atoms)} missing atoms")
print(f"{len(missing_terminals)} missing terminals")

if len(nonstandard_residues)>0:
    fixer.replaceNonstandardResidues()

if len(missing_atoms)>0:
    fixer.addMissingAtoms()

0 missing residues
0 non standard residues
6 missing atoms
0 missing terminals


In [17]:
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')

In [18]:
modeller = app.Modeller(fixer.topology, fixer.positions)
pH = 7.2
residues_protonated = modeller.addHydrogens(forcefield=forcefield, pH=pH)

In [19]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

In [20]:
# We need to define force groups to be able to calculate the different energy terms 

forcegroups = {}
for ii in range(system.getNumForces()):
    force = system.getForce(ii)
    print(f'{force.getName()} was in force group {force.getForceGroup()} and it is now in group {ii}')
    force.setForceGroup(ii)
    forcegroups[force] = ii

HarmonicBondForce was in force group 0 and it is now in group 0
HarmonicAngleForce was in force group 0 and it is now in group 1
NonbondedForce was in force group 0 and it is now in group 2
PeriodicTorsionForce was in force group 0 and it is now in group 3
CMMotionRemover was in force group 0 and it is now in group 4


In [21]:
# A simulation object is needed

## Integrator
step_size   = 0.002*unit.picoseconds
temperature = 0.0*unit.kelvin
friction    = 0.0/unit.picosecond # Damping para la dinámica de Langevin

integrator = mm.LangevinIntegrator(temperature, friction, step_size)

## Platform
platform_name = 'CPU'
platform    = mm.Platform.getPlatformByName(platform_name)

In [22]:
# Creación del objeto context del sistema en vacio

context = mm.Context(system, integrator, platform)

In [23]:
# Condiciones iniciales
context.setPositions(modeller.positions)

In [24]:
# Minimizacion del sistema

state_pre_minimization = context.getState(getEnergy=True)
mm.LocalEnergyMinimizer_minimize(context)
state_post_minimization = context.getState(getEnergy=True)

In [25]:
print('Energy before minimization:', state_pre_minimization.getPotentialEnergy())
print('Energy after minimization:', state_post_minimization.getPotentialEnergy())

Energy before minimization: -13298.469521243596 kJ/mol
Energy after minimization: -24997.509341596022 kJ/mol


In [26]:
energies = {}
for ff, ii in forcegroups.items():
    energies[ff.getName()] = context.getState(getEnergy=True, groups={ii}).getPotentialEnergy()

In [27]:
for ii,jj in energies.items():
    print(ii,jj)

HarmonicBondForce 412.43904571880404 kJ/mol
HarmonicAngleForce 1692.1235454847406 kJ/mol
NonbondedForce -36629.51595539581 kJ/mol
PeriodicTorsionForce 9527.444022596237 kJ/mol
CMMotionRemover 0.0 kJ/mol


In [28]:
energies['HarmonicBondForce']+energies['HarmonicAngleForce']+energies['NonbondedForce']+energies['PeriodicTorsionForce']

Quantity(value=-24997.509341596022, unit=kilojoule/mole)

- We should work on how to decompose de NonbondedForce: LJ contribution (VdW) and Coulomb contribution. 

In [29]:
non_bonded_force = None
for ii in range(system.getNumForces()):
    force = system.getForce(ii)
    if force.getName() == 'NonbondedForce':
        non_bonded_force = force

In [30]:
non_bonded_force.getName()

'NonbondedForce'

In [31]:
non_bonded_force.getNumParticles()

3159

In [32]:
non_bonded_force.getNumExceptions()

17259

In [33]:
original_particle_parameters = {}
for ii in range(non_bonded_force.getNumParticles()):
    charge, sigma, epsilon = non_bonded_force.getParticleParameters(ii)
    original_particle_parameters[ii] = {'charge': charge, 'sigma': sigma, 'epsilon':epsilon}

original_exception_parameters = {}
for ii in range(non_bonded_force.getNumExceptions()):
    particle1, particle2, chargeProd, sigma, epsilon = non_bonded_force.getExceptionParameters(ii)
    original_exception_parameters[ii] = {'particle1': particle1, 'particle2': particle2, 'chargeProd': chargeProd,
                                     'sigma': sigma, 'epsilon': epsilon}

In [34]:
residues_per_chain={}
atoms_per_residue={}
for chain in modeller.topology.chains():
    aux_residue_indices = []
    for residue in chain.residues():
        aux_atom_indices = []
        aux_residue_indices.append(residue.index)
        for atom in residue.atoms():
            aux_atom_indices.append(atom.index)
        atoms_per_residue[residue.index]=aux_atom_indices
    residues_per_chain[chain.index]=aux_residue_indices

In [35]:
Total_non_bonded_potential_energy = context.getState(getEnergy=True, groups={2}).getPotentialEnergy()

In [36]:
Total_non_bonded_potential_energy

Quantity(value=-36629.51595539581, unit=kilojoule/mole)

In [37]:
original_particle_parameters[0]

{'charge': Quantity(value=0.1414, unit=elementary charge),
 'sigma': Quantity(value=0.3249998523775958, unit=nanometer),
 'epsilon': Quantity(value=0.7112800000000001, unit=kilojoule/mole)}

In [38]:
residues_in = residues_per_chain[0]
atoms_in = []
for residue_index in residues_in:
    atoms_in += atoms_per_residue[residue_index]

Let's compute the energy terms of the system when all charges and epsilons have been set to zero:

In [70]:
for ii, parameters in original_particle_parameters.items():
    non_bonded_force.setParticleParameters(ii, 0.0 * unit.elementary_charge, parameters['sigma'],
                                           0.0 * unit.kilojoule_per_mole)

for ii, parameters in original_exception_parameters.items():
    non_bonded_force.setExceptionParameters(ii, parameters['particle1'], parameters['particle2'],
                                            0.000000001 * unit.elementary_charge **2, parameters['sigma'],
                                            0.0 * unit.kilojoule_per_mole)
    
non_bonded_force.updateParametersInContext(context)

energies_zeroed = {}
for ff, ii in forcegroups.items():
    energies_zeroed[ff.getName()] = context.getState(getEnergy=True, groups={ii}).getPotentialEnergy()

for ii,jj in energies_zeroed.items():
    print(ii,jj)

OpenMMException: updateParametersInContext: The number of non-excluded exceptions has changed

In [53]:
original_exception_parameters[11]

{'particle1': 1,
 'particle2': 5,
 'chargeProd': Quantity(value=0.014794441666666667, unit=elementary charge**2),
 'sigma': Quantity(value=0.1514527820838577, unit=nanometer),
 'epsilon': Quantity(value=0.032844399999999996, unit=kilojoule/mole)}

In [68]:
non_bonded_force.setExceptionParameters(11, 1, 5,
                                        0.000000001 * unit.elementary_charge **2, 0.1 * unit.nanometer, 0.0 * unit.kilojoule_per_mole)

In [69]:
non_bonded_force.updateParametersInContext(context)

energies_zeroed = {}
for ff, ii in forcegroups.items():
    energies_zeroed[ff.getName()] = context.getState(getEnergy=True, groups={ii}).getPotentialEnergy()

for ii,jj in energies_zeroed.items():
    print(ii,jj)

HarmonicBondForce 412.43904571880404 kJ/mol
HarmonicAngleForce 1692.1235454847406 kJ/mol
NonbondedForce 37677.480324850025 kJ/mol
PeriodicTorsionForce 9527.444022596237 kJ/mol
CMMotionRemover 0.0 kJ/mol


# Of interest

https://github.com/openmm/openmm/issues/1463    
https://github.com/openmm/openmm/issues/387    
https://github.com/openmm/openmm/issues/2682    
https://github.com/openmm/openmm/issues/1830    
https://openmm.github.io/openmm-cookbook/dev/notebooks/cookbook/Analyzing%20Energy%20Contributions.html    
https://openmm.github.io/openmm-cookbook/dev/notebooks/cookbook/Querying%20Charges%20and%20Other%20Parameters.html    


https://github.com/openmm/openmm/issues/3178    
https://github.com/openmm/openmm/issues/3169    
https://github.com/openmm/openmm/issues/2880    
https://github.com/openmm/openmm/issues/2835    
https://github.com/openmm/openmm/issues/2682    

https://chemrxiv.org/engage/api-gateway/chemrxiv/assets/orp/resource/item/60c758a8567dfe4abcec68b3/original/sbm-open-mm-a-builder-of-structure-based-models-for-open-mm.pdf